In [1]:
s = [' San Francisco Giants Season ticket holder many available! 5th ROW!',
    'Super Bowl XIX 49er Rare Ticket','Bear Valley Lift Ticket']
import spacy

ImportError: No module named spacy

In [25]:
%%time
#
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy import en
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.grid_search import GridSearchCV

vectorizer=TfidfVectorizer(ngram_range=(1,3), 
                         stop_words=en.STOPWORDS,token_pattern=r'\w+')
X = vectorizer.fit_transform(train)

param_grid = [{'n_clusters':range(100,200)}]
km = KMeans(init='k-means++', max_iter=100, n_init=1)

gs = GridSearchCV(km, param_grid=param_grid,n_jobs=-2)#,scoring=metrics.silhouette_score)

gs.fit(X)

#print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(X, km.labels_, sample_size=1000))



CPU times: user 1h 1min 45s, sys: 4min 3s, total: 1h 5min 48s
Wall time: 9min 21s


In [1]:
%%time
#import data
from spacy.en import English
nlp = English()

path = '/home/vagrant/datacourse/fellow_only_material/Capstone/'
train = []
for line in open(path + 'training.txt','r'):
    train.append(line[:-1])

CPU times: user 8.96 s, sys: 3.05 s, total: 12 s
Wall time: 12 s


In [2]:
%%time
#get vocabulary

doc = nlp(' '.join(train).decode('utf8'))

ents = []

for ent in doc.ents:
    if ent.label_ != 'CARDINAL':
        ents.append((ent.text.lower(),ent.label_))

vocab = set(ents)



CPU times: user 1.79 s, sys: 20.8 ms, total: 1.81 s
Wall time: 1.81 s


In [26]:
# sim functions
import re, math
from collections import Counter

WORD = re.compile(r'\w+')
NUM = re.compile(r'\d+')

def get_cosine(text1, text2, weights, threshold):
    vec1 = WORD.findall(text1)
    vec2 = WORD.findall(text2)
    
    w1 = sumW(vec1,weights)
    w2 = sumW(vec2,weights)
    denominator = math.sqrt(w1*w2)
    if w1 < w2:
        if w1 < denominator*threshold: return 0.0
    elif w2 < denominator*threshold: return 0.0
    if not denominator: return 0.0
    
    intersection = set(vec1) & set(vec2)
    numerator = sumW(intersection, weights)
    
    return float(numerator) / denominator

def sumW(vec, weights):
    s = 0.0
    for w in vec:
        if not NUM.match(w):
            s += 1.0/weights.get(w,1.0)
    return s


In [8]:
%%time
#dedup vocab
import pandas as pd
import sklearn as sk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy import en
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.grid_search import GridSearchCV

df = pd.DataFrame(list(vocab),columns=['ent','type'])
df['key'] = 1
merged = pd.merge(df,df,on='key').drop_duplicates()
merged = merged[(merged.type_x == merged.type_y) & (merged.type_x != 'DATE')]

threshold = 0.7
vectorizer=TfidfVectorizer(token_pattern=r'\w+')   
X = vectorizer.fit_transform(train)
weights = {x[0]:x[1] for x in zip(vectorizer.get_feature_names(),vectorizer.idf_)}

merged['sim'] = merged.apply(lambda row: get_cosine(row.ent_x,row.ent_y,weights,threshold),axis=1)



CPU times: user 16.5 s, sys: 171 ms, total: 16.6 s
Wall time: 16.6 s


In [9]:
dedup = merged[merged.sim >= threshold]
dups = dedup[dedup.ent_x != dedup.ent_y]
to_replace = {}

for i,row in dups.iterrows():
    small = row.ent_x
    large = row.ent_y  
    if len(row.ent_x) > len(row.ent_y):
        small, large = large, small
    to_replace[small] = large

In [10]:
import networkx as nx

G = nx.Graph()
for i,j in to_replace.iteritems():
    G.add_edge(i,j)
sets = [c for c in sorted(nx.connected_components(G), key=len, reverse=True)]
sets

[{u'jack',
  u'jack u',
  u'jack u - 2',
  u'jack u - wednesday',
  u'jack u commuter',
  u'jack u concert',
  u'jack u ga',
  u'jack u tonight',
  u'jack u tuesday',
  u'jack u wed',
  u'jack u wednesday',
  u'jack \xfc',
  u'jack \xfc tuesday',
  u'jack-u'},
 {u'coachella - weekend',
  u'coachella 1',
  u'coachella weekend',
  u'coachella weekend 1 shuttle',
  u'coachella weekend 2',
  u'coachella weekend 2 ga',
  u'coachella weekend 2 tickets + shuttle passes - in',
  u'coachella weekend 2 tix',
  u'fillmore coachella weekend',
  u'fv coachella weekend',
  u'memphis coachella weekend'},
 {u'105 row',
  u'106 row',
  u'108 row',
  u'123 row',
  u'124 row',
  u'207 row',
  u'328 row'},
 {u'section 122',
  u'section 145',
  u'section 147',
  u'section 149',
  u'section 224',
  u'section 232'},
 {u'grizzlies row',
  u'grizzlies row 5',
  u'memphis grizzlies',
  u'memphis grizzlies lower level row 5',
  u'memphis grizzlies row',
  u'row 8'},
 {u'coachella ticket weekend',
  u'coachella w

In [15]:
from collections import Counter
def most_common(c,n):
    a = Counter([w for st in c for w in WORD.findall(st)]).most_common(n)
    return ' '.join(map(lambda x: x[0],a))
#map(lambda s: min(s),sets)
{w:min(s) for s in sets for w in s if w != min(s)}

{u'10 7:15 p.m. plb': u'10 7:15 p.m.',
 u'106 row': u'105 row',
 u'108 row': u'105 row',
 u'123 row': u'105 row',
 u'124 row': u'105 row',
 u'20-$60/ticket': u'18-$90/ticket',
 u'207 row': u'105 row',
 u'3 or 4 seats': u'2 seats',
 u'3 seat': u'14 seat',
 u'30-$140/ticket': u'18-$90/ticket',
 u'328 row': u'105 row',
 u'4 seats': u'4 club seats & 1',
 u'4 warriors playoff': u'4 centre court warriors playoff round',
 u'4/13 show': u'4/12 show',
 u'4/23 two coachella weekend': u'2x coachella weekend',
 u'4800.00 2 coachella shuttle': u'2 2 coachella',
 u'adele &more': u'adele',
 u'at&t park april': u'at&t park',
 u'at&t park cubs': u'at&t',
 u'at&t park regal': u'at&t',
 u'ben harper aug': u'ben harper',
 u'beyonce floor': u'beyonce',
 u'bill graham - saturday': u'bill graham',
 u'bill graham auditorium': u'bill graham',
 u'boston red sox (': u'boston red sox',
 u'box electric daisy carnival': u'box electric daisy',
 u'c212 row': u'bl row 0',
 u'chris martin': u'chris',
 u'coachella 1': u

In [39]:
class Dedup_transformer(sk.base.BaseEstimator, sk.base.TransformerMixin):
    def __init__(self, dedup_map):
        self.dedup_map = dedup_map 

    def fit(self, X): 
        return self

    def transform(self, X):
        #X is list of str
        return map(self.replace_dups,X)
    
    def replace_dups(self, st):
        ents = nlp(st.decode('utf8')).ents
        joined = st.decode('utf8')
        for ent in ents:
            joined = joined.replace(ent.text, self.dedup_map.get(ent.text.lower(),ent.text))
        return joined.lower()

In [127]:
st = u'San Jose Earthquakes - 04/13 vs New York Red Bulls at 7:30PM'
for i in nlp(st.decode('utf8')).ents:
    print i.text, i.label_

San Jose GPE


In [5]:
#modified Tfidf transformer uses df instead of idf
import numpy as np
import scipy.sparse as sp
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.utils.fixes import frombuffer_empty, bincount

class ModTfidfTransformer(TfidfTransformer):

    def fit(self, X, y=None):
        """Learn the idf vector (global term weights)
        Parameters
        ----------
        X : sparse matrix, [n_samples, n_features]
            a matrix of term/token counts
        """
        if not sp.issparse(X):
            X = sp.csc_matrix(X)
        if self.use_idf:
            n_samples, n_features = X.shape
            df = _document_frequency(X)

            # perform idf smoothing if required
            df += int(self.smooth_idf)
            n_samples += int(self.smooth_idf)

            # log+1 instead of log makes sure terms with zero idf don't get
            # suppressed entirely.
            temp = np.log(float(n_samples) / df) + 1.0
            mean = np.mean(temp)
            #gives high value for very common and uncommon words
            idf = 1.0/temp#np.maximum.reduce([temp,1.0/temp])
            #print idf
            self._idf_diag = sp.spdiags(idf,
                                        diags=0, m=n_features, n=n_features)

        return self
    
def _document_frequency(X):
    #"""Count the number of non-zero values for each feature in sparse X."""
    if sp.isspmatrix_csr(X):
        return bincount(X.indices, minlength=X.shape[1])
    else:
        return np.diff(sp.csc_matrix(X, copy=False).indptr)

 

In [40]:
dedup = Dedup_transformer(to_replace)
s = ['ABC abc bbb ccc','one or Bart yes!']
dedup.transform(s)

[u'abc abc bbb ccc', u'one or bart tix - $ yes!']

In [130]:
import re
r = re.compile(r'(?u)\b[A-Z]+\w*\s+[A-Z]+\w*\b')
r1 = re.compile(r"(?u)\b[A-Z]\w+\b")
s = u"(2x) abcA Coachella Weekend 2 GA Tickets Adele"
r.findall(s)

[u'Coachella Weekend', u'GA Tickets']

In [32]:
train[:5]

['1-2 NOFX TICKETS TONIGHT E-MAILED BERKELEY UC THEATRE',
 'Selling 1 Coachella Weekend 2 GA Ticket Wristband',
 'Chase Bank $500 coupon',
 '****JACK U CONCERT TONIGHT****',
 '**WANTED** coachella weekend 1']

In [138]:
%%time
#find optimal clusters
from sklearn import pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.grid_search import GridSearchCV
from spacy import en
import re

TOK = re.compile(r"(?u)\b[A-Z]\w+\b")
def tokenizer(s): 
    return [x.lower() for x in TOK.findall(s)]

stop = ['tickets','ticket','row','seat','seats','lower','tix','level','aisle']

km = KMeans(init='k-means++', max_iter=100, n_init=1)
voc = set([x[0].lower() for x in vocab])
def ent_tok(st):
    doc = nlp(st)
    return [x.text.lower() for x in doc.ents]

pipe = pipeline.Pipeline([
        #('dedup', Dedup_transformer(to_replace)),
        ('vectorizer', CountVectorizer(min_df=2,
                                       ngram_range=(1,3),
                                       stop_words=en.STOPWORDS.union(stop),
                                       tokenizer=tokenizer,
                                       lowercase=False)),
        ('transformer', ModTfidfTransformer()),
        ('kmeans',km)
    ])
#142
params = dict(kmeans__n_clusters= [20,21])

grid = GridSearchCV(pipe,param_grid=params,n_jobs=-1)

grid.fit(train)
#km.fit(X)

CPU times: user 1.13 s, sys: 219 ms, total: 1.35 s
Wall time: 1.84 s


In [118]:
grid.best_params_

{'kmeans__n_clusters': 248}

In [64]:
print grid.best_estimator_.named_steps['vectorizer'].vocabulary == voc
print len(voc)

False
1012


In [19]:
idfs = grid.best_estimator_.named_steps['transformer'].idf_
a = []
for w, i in grid.best_estimator_.named_steps['vectorizer'].vocabulary_.iteritems():
    a.append((w,idfs[i]))

sorted(a,key=lambda x: x[1],reverse=True)

[(u'coachella', 0.35978462498556313),
 (u'warriors', 0.35824834640534808),
 (u'weekend', 0.32452331046862748),
 (u'giants', 0.3163070068108027),
 (u'playoff', 0.27282221746342816),
 (u'grizzlies', 0.26266256194062765),
 (u'game', 0.25801883577523538),
 (u'shuttle', 0.25754586804882856),
 (u'sf', 0.25563737644984824),
 (u'ga', 0.25418794965210595),
 (u'pass', 0.24872992987539991),
 (u'jack', 0.2430183424693472),
 (u'san', 0.24248505090981273),
 (u'sharks', 0.2364442894467752),
 (u'golden', 0.23416083215207315),
 (u'playoffs', 0.23358203559928881),
 (u'state', 0.23358203559928881),
 (u'round', 0.23123331992629445),
 (u'club', 0.22760227819615428),
 (u'passes', 0.22698356449702553),
 (u'box', 0.22698356449702553),
 (u'sec', 0.2218791567456711),
 (u'beyonce', 0.21853382346129802),
 (u'selling', 0.21288773168286096),
 (u'april', 0.21288773168286096),
 (u'oakland', 0.21215332784742846),
 (u'fox', 0.21141195971540871),
 (u'day', 0.21141195971540871),
 (u'jose', 0.20914399886905807),
 (u'face'

In [17]:
# get top terms per cluster
order_centroids = grid.best_estimator_.named_steps['kmeans'].cluster_centers_.argsort()[:, ::-1]
terms = grid.best_estimator_.named_steps['vectorizer'].get_feature_names()
for i in range(299):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :5]:
        print(' %s' % terms[ind])
    print()

Cluster 0:
 david
 adam
 sandler
 spade
 floor
()
Cluster 1:
 warriors
 grizzlies
 memphis
 sec
 secti
()
Cluster 2:
 coachella
 weekend
 ga
 car
 camping
()
Cluster 3:
 giants
 sf
 event
 special
 marlins
()
Cluster 4:
 jack
 ticketmaster
 french
 game
 ga
()
Cluster 5:
 playoff
 sharks
 great
 face
 sj
()
Cluster 6:
 day
 outside
 pass
 lands
 passes
()
Cluster 7:
 fox
 face
 value
 theater
 m83
()
Cluster 8:
 warriors
 playoff
 round
 game
 sec
()
Cluster 9:
 giants
 sec
 games
 chicago
 plb
()
Cluster 10:
 beyonce
 vip
 tour
 formation
 world
()
Cluster 11:
 sec
 acoustic
 zhu
 fresno
 game
()
Cluster 12:
 giants
 san
 club
 francisco
 season
()
Cluster 13:
 magazine
 pop
 sold
 tonight
 orchestra
()
Cluster 14:
 warriors
 sec
 play
 san
 round
()
Cluster 15:
 coachella
 weekend
 vip
 sale
 wristband
()
Cluster 16:
 ballet
 mana
 international
 bryan
 russell
()
Cluster 17:
 jack
 graham
 tuesday
 ga
 auditorium
()
Cluster 18:
 coachella
 shuttle
 pass
 wknd
 admission
()
Cluster 1

In [18]:
s = 'Parov Stelar Tickets (2)'
for ind in order_centroids[grid.predict([s])[0],:5]:
    print terms[ind]

stelar
parov
zhu
fresno
game


In [ ]:
#Step 1: extract dates ()
#Step 2: strip out numbers
#Step 3: update list of stop words
#Step 4: Extract entities
#Step 5: ask user to dedup common entities
#Step 6: retrain and loop starting from step 3

In [2]:
#Step 1: extract dates ()
from datetime import datetime
import time
from datefinder import DateFinder

dfin = DateFinder()

dates = []
clean = []
#disambiguate dates TODO

for x in train:
    matches = dfin.find_dates(x)
    date = [y for y in matches if y.year >= datetime.fromtimestamp(time.time()).year]
    d = ''
    if len(date) == 1:
        d = dfin.extract_date_strings(x).next()[0]
    clean.append(x.replace(d,''))
    dates.append(date)



/opt/conda/lib/python2.7/site-packages/dateparser/languages/dictionary.py:84: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  (token in ALWAYS_KEEP_TOKENS) or


In [3]:
#Step 2: strip out numbers
import re
num = re.compile(r'\b\d+\b',re.U)
w = re.compile(r'\w+')

cleaner = []
for s in clean:
    new = ' '.join(x for x in w.findall(s) if not num.search(x))
    try:
        cleaner.append(new)
    except:
        print new


In [4]:
#Step 3: update list of stop words
sw = ['tickets','ticket','row','seat','seats','lower',
      'tix','level','aisle']

In [39]:
#Step 4: add labels to some data
path = '/home/vagrant/datacourse/fellow_only_material/Capstone/'
labels = []
raw_labels = []

for line in open(path + 'labels_train.txt','r'):
    labels.append(line[:-1])
    raw_labels.append(line[:-1])
for i in range(len(train) - len(labels)):
    labels.append(-1)

In [8]:
class DenseTransformer(TransformerMixin):

    def transform(self, X, y=None, **fit_params):
        return X.todense()

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

    def fit(self, X, y=None, **fit_params):
        return self

In [9]:
from sklearn.preprocessing import LabelEncoder
from numpy import copy

enc = LabelEncoder()
encoded = enc.fit_transform(labels)

semi_labeled = copy(encoded)
semi_labeled[encoded==0] = -1

semi_labeled

array([23,  8,  6, ..., -1, -1, -1])

In [20]:
s = 'Sharks Playoff vs Kings Glass 49ers VIP Tix - Sharks Shoot Twice!'
tokenizer(s)
#re.findall(r'\w+',s)

def spacy_tokenizer(s):
    sp = nlp(unicode(s))
    return [x.text for x in sp.ents]

TOK = re.compile(r"(?u)\b[0-9A-Z]\w+\b")
def tokenizer(s): 
    return [x.lower() for x in TOK.findall(s)]

tokenizer(s)

['sharks',
 'playoff',
 'kings',
 'glass',
 '49ers',
 'vip',
 'tix',
 'sharks',
 'shoot',
 'twice']

In [21]:
%%time
#Step 5: train semi-supervised model
from sklearn.semi_supervised import LabelPropagation
from sklearn import pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.grid_search import GridSearchCV
from spacy import en
import re

TOK = re.compile(r"(?u)\b[0-9A-Z]\w+\b")
def tokenizer(s): 
    return [x.lower() for x in TOK.findall(s)]

semi = LabelPropagation('knn',n_neighbors=1)

pipe = pipeline.Pipeline([
        #('dedup', Dedup_transformer(to_replace)),
        ('vectorizer', CountVectorizer(min_df=1,
                                       ngram_range=(1,1),
                                       stop_words=en.STOPWORDS.union(sw),
                                       tokenizer=tokenizer,
                                       lowercase=False)),
        ('transformer', ModTfidfTransformer()),
        ('dense',DenseTransformer()),
        ('semi',semi)
    ])

X_train = cleaner
y_train = semi_labeled
pipe.fit(X_train, y_train)

CPU times: user 4.84 s, sys: 7.39 ms, total: 4.85 s
Wall time: 4.85 s


In [22]:
enc.classes_[pipe.named_steps['semi'].transduction_][80:100]

array(['Caetano Veloso and Gilberto Gil', "Angels vs Oakland A's",
       'Earthquakes vs Red Bulls', 'Coachella', 'Coachella',
       'Ariana Huffington', 'Warriors vs Grizzlies', '49ers', 'Coachella',
       '49ers', "Angels vs Oakland A's", 'North Sierra Wine Trail',
       'Phish', 'Giants vs Marlins', '49ers', '49ers', '49ers', '49ers',
       '49ers', 'Coachella Shuttle'], 
      dtype='|S32')

In [23]:
enc.classes_[pipe.predict(['Sharks Playoff vs Kings Glass VIP Tix - Sharks Shoot Twice!'])]

array(['49ers'], 
      dtype='|S32')

In [10]:
train[50:60]

['Memphis May Fire tickets - The Regency Ballroom 5/11 - Discounted!',
 "Karl Denson's Tiny Universe tickets - The Fillmore 5/7 - Discounted!",
 '49ERS Jerry Rice SIGNED RED JERSEY',
 '49ers 2/3/5 tickets row 2',
 'Raiders BENCH 6 lowers 50YD LINE',
 'Warriors Rd 2 Playoff Tix',
 '49er 2016 season ALL GAMES - UNDER FACE VALUE',
 "Dru Hill tickets - Yoshi's 5/6 - Discounted!",
 'Thao & The Get Down Stay Down tickets - The Fillmore 4/30 - Discounted',
 'Discounted - AMON AMARTH tickets - Warfield 5/20']

In [1]:
%%time
#import data
#from spacy.en import English
#nlp = English()

path = '/home/vagrant/datacourse/fellow_only_material/Capstone/'
train = []
for line in open(path + 'training.txt','r'):
    train.append(line[:-1])
    
real_events = []
for line in open(path + '2016042800-2016072800.txt'):
    real_events.append(line[:-1].split('::'))

CPU times: user 10.5 ms, sys: 11.5 ms, total: 22 ms
Wall time: 19.8 ms


In [24]:
dfin = DateFinder()
s_dates = ' '.join('%.4d%.2d%.2d' % (dt.year,dt.month,dt.day) for dt in dfin.find_dates('the event is this Today'))
s_dates

''

In [2]:
#cosine similarity approach
#step1: predict labels based on cosine sim
from collections import defaultdict, OrderedDict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from spacy import en
import re, math
from collections import Counter
import numpy as np

from datetime import datetime
import time
from datefinder import DateFinder



class Sim_predictor():
    def __init__(self):
        self.weights = defaultdict(float)
        self.classes = []
        self.dates = []
        self.venues = []
        self.X = []
        self.WORD = re.compile(r'\w+')
        self.NUM = re.compile(r'\d+')


    
    #X is list of (str,YYYYMMDD)
    def fit(self, X, stop_words,K):
        self.dates = [z[1] for z in X]
        self.venues = [z[2] for z in X]
        self.classes = [z[0] for z in X]

        X = [' '.join(l) for l in X]
        self.X = X
        vectorizer = CountVectorizer()
        tfidf = TfidfTransformer()   
        #X = tfidf.fit_transform(vectorizer.fit_transform(train))
        #weights = {x[0]:x[1] for x in zip(vectorizer.get_feature_names(),tfidf.idf_)}
        w1 = {}
        w2 = {}
        for i,title in enumerate(self.classes):
            weight = (len(X) - i)**2
            for w in self.WORD.findall(title):
                w = w.lower()
                if w not in w1:
                    w1[w] = weight
                else:
                    w1[w] += weight
        Z = tfidf.fit_transform(vectorizer.fit_transform(self.classes))
        for w,idf in zip(vectorizer.get_feature_names(),tfidf.idf_):
            w2[w] = idf
        
        mean1, std1 = np.mean(w1.values()),np.std(w1.values())
        mean2, std2 = np.mean(w2.values()),np.std(w2.values())
        #popularity importance
        
        for k,v in w1.iteritems():
            self.weights[k] = K*math.exp((v-mean1)/std1)
        for k,v in w2.iteritems():
            self.weights[k] += (1-K)*math.exp((v-mean2)/std2)
            

            
        #dates are important
        for d in self.dates:
            self.weights[d] = 1
        #venues are important
        for v in self.venues:
            for w in v.split():
                self.weights[w] = 1
        for w in stop_words:
            self.weights[w] = 0.0
        
            
    def transform(self,X):
        return self
    
    def predict(self,X, threshold):
        dfin = DateFinder()
        pred_labels = []

        for s in X:
            s_dates = ' '.join('%.4d%.2d%.2d' % (dt.year,dt.month,dt.day) for dt in dfin.find_dates(s))
            date_str = dfin.extract_date_strings(s)
            #print s_dates
            for st in date_str:
                #print st[0],
                s = s.replace(st[0],'')
            s = ' '.join([s,s_dates])
            m = (None,0)
            #print s.lower()
            for i, label in enumerate(self.X):
                #date = self.dates[i]
                
                d1 = self.get_cosine(s.lower(), label.lower(),self.weights,threshold)
                #print d1,label
                #d2 = max(1,(date in s_dates)*2)
                if d1 > m[1]:
                    m = (label, d1)
            pred_labels.append((s,m))
        return pred_labels
        
    def get_cosine(self, text1, text2, weights, threshold):
        vec1 = self.WORD.findall(text1)
        vec2 = self.WORD.findall(text2)

        w1 = self.sumW(vec1,self.weights)
        w2 = self.sumW(vec2,self.weights)
        denominator = math.sqrt(w1*w2)
        if w1 < w2:
            if w1 < denominator*threshold: return 0.0
        elif w2 < denominator*threshold: return 0.0
        if not denominator: return 0.0

        intersection = set(vec1) & set(vec2)
        numerator = self.sumW(intersection, self.weights)
        # if only dates match, ignore
        if len(intersection) == 1:
            if self.NUM.search(list(intersection)[0]):
                return 0.0
        else:
            return float(numerator) / denominator

    def sumW(self, vec, weights):
        s = 0.0
        for w in vec:
            #ignore numbers
            #if not self.NUM.match(w):
            s += self.weights.get(w,0.0001)
        return s

    def get_overlap(self, text1, text2, weights):
        vec1 = self.WORD.findall(text1)
        vec2 = self.WORD.findall(text2)
        intersection = set(vec1) & set(vec2)
        return self.sumW(intersection,weights)


    

model = Sim_predictor()
sw = en.STOPWORDS.union(set(['ticket','tickets','san','francisco','s']))
model.fit(real_events,sw,0.01)



#get_cosine(s, 'Sharks vs Kings',weights,threshold)

In [63]:
t1 = 'Outside Lands Festival- tickets,  20160429 20160805'.lower()
t2 = 'Friday Happy Hour 20160429 New Leaf - Pleasanton'.lower()
model.get_cosine(t1,t2,model.weights,threshold)

0.0

In [67]:
p = train
threshold = 0.01
predicted = model.predict(p, threshold) 

In [69]:
print len(predicted)
print len([x for x in predicted if x[1][1] >= threshold])

2500
1643


In [74]:
threshold = 0.4
threshold2 = 0.6
sorted([x for x in predicted if x[1][1] >= threshold and x[1][1] < threshold2],key=lambda x: x[1][1],reverse=False)

[('2 Tickets  Drake Concert @ Oracle ',
  ('NBA Western Conference Finals: TBD at Golden State Warriors - Home Game 2 (If Necessary - Date TBD) 20160518 Oracle Arena',
   0.4000883732630534)),
 ('Giants vs Padres  tickets sec 38 $5 each PayPal only 4/6 03280429 20160426',
  ('Musical Theatre- Wednesdays 4:30-6:00 20160428 Young Performers Theatre',
   0.4001774092287461)),
 ('Giants vs Padres  tickets sec 38 $5 each PayPal only 4/6 03280429 20160426',
  ('Musical Theatre- Wednesdays 4:30-6:00 20160428 Young Performers Theatre',
   0.4001774092287461)),
 ('WARRIORS!!!! GAME  - HALF COURT - THREE (3) TICKETS ',
  ('NBA Western Conference Finals: TBD at Golden State Warriors - Home Game 3 (If Necessary - Date TBD) 20160526 Oracle Arena',
   0.40059580924292587)),
 ("COLDPLAY -- Levi's Stadium FIELD TICKETS --  20160903",
  ('100 Years of CAL Boxing 20160507 Kabam Field at California Memorial Stadium',
   0.4007580084566795)),
 ('SF Giants v Boston Red Sox -  - Premium Lower Box w Parking 

In [16]:
[x for x in real_events if '20160429' in x[1]]

[['Luke Bryan with Little Big Town and Dustin Lynch',
  '20160429',
  'Concord Pavilion'],
 ['Black Mountain', '20160429', 'The Independent'],
 ["Tony Visconti + Woody Woodmansey's Holy Holy - Celebrating David Bowie",
  '20160429',
  'The Fillmore'],
 ['Floating Points (Live)', '20160429', 'Mezzanine'],
 ['Washington Huskies at Cal Bears Baseball', '20160429', 'Evans Diamond'],
 ['Tierra', '20160429', 'Social Hall SF'],
 ['Morgan Page', '20160429', 'Temple Nightclub'],
 ['RYAN HEMSWORTH + NEF THE PHARAOH at 1015 FOLSOM',
  '20160429',
  '1015 Folsom'],
 ['Stephan Bodzin (LIVE), HVOB (LIVE), Satori Live (Krooks, Netherlands)',
  '20160429',
  'Public Works'],
 ['OMI', '20160429', 'Regency Ballroom'],
 ['SF BEER (and WINE) OLYMPICS! Drinking Games + $2 Beer/Wine + $2.50 House Spirits! Fridays!',
  '20160429',
  'Horizon Nightclub'],
 ['Underground Fridays @ Asia SF', '20160429', 'AsiaSF'],
 ["04/29/16 Fast Times Dance Party Stars Rock Dan's Bar",
  '20160429',
  "Dan's Bar"],
 ['WAH! Fe

In [26]:
train[200:210]

['Admission to Gilroy Gardens',
 '4 tix for Pier 39 tours, attractions, 2-hours free garage parking',
 'Tickets to Walt Disney Family Museum in SF Presidio',
 '2 Boarding Passes for 60 minutes Blue&Gold Bay Cruise - half price',
 'United Airline Roundtrip flight - up to 20% off on ual.com',
 'WARRIORS VS ROCKETS LOWER LEVEL TICKETS ROW 13! LESS THAN STUBHUB!!',
 'Megabus ticket SF to LA for 4/28 at 10:30pm',
 'WARRIORS PLAYOFFS ALL ROUNDS 6 lowers',
 '4 x Bottlerock General Admission 3 Day Passes',
 'Warriors tickets']

In [7]:
# sim functions
import re, math
from collections import Counter

WORD = re.compile(r'\w+')
NUM = re.compile(r'\d+')

def get_cosine(text1, text2, weights, threshold):
    vec1 = WORD.findall(text1)
    vec2 = WORD.findall(text2)
    
    w1 = sumW(vec1,weights)
    w2 = sumW(vec2,weights)
    denominator = math.sqrt(w1*w2)
    if w1 < w2:
        if w1 < denominator*threshold: return 0.0
    elif w2 < denominator*threshold: return 0.0
    if not denominator: return 0.0
    
    intersection = set(vec1) & set(vec2)
    numerator = sumW(intersection, weights)
    
    return float(numerator) / denominator

def sumW(vec, weights):
    s = 0.0
    for w in vec:
        #ignore numbers
        if not NUM.match(w):
            s += weights.get(w,0.0001)
    return s

def get_overlap(text1, text2, weights):
    vec1 = WORD.findall(text1)
    vec2 = WORD.findall(text2)
    intersection = set(vec1) & set(vec2)
    return sumW(intersection,weights)

In [41]:
[x for x in zip(vectorizer.get_feature_names(),tfidf.idf_) if x[0] =='san']

[(u'san', 3.3731152437998837)]

In [43]:
sorted([(k,v) for k,v in weights.iteritems()],key=lambda x: x[1],reverse=1)

[('dojo', 3693.5849624745006),
 ('year', 3693.5849624745006),
 ('expo', 3686.953750692679),
 ('decorator', 3680.3225389108579),
 ('surf', 3673.6913271290364),
 ('prism', 3673.6913271290364),
 ('tats', 3673.6913271290364),
 ('nada', 3673.6913271290364),
 ('lizard', 3667.0601153472153),
 ('murlocs', 3667.0601153472153),
 ('gizzard', 3667.0601153472153),
 ('king', 3667.0601153472153),
 ('wizard', 3667.0601153472153),
 ('caifanes', 3660.4289035653937),
 ('regional', 3653.7976917835722),
 ('caves', 3653.7976917835722),
 ('vasco', 3653.7976917835722),
 ('fit', 3647.1664800017511),
 ('afterschool', 3647.1664800017511),
 ('berthiaume', 3640.5352682199295),
 ('denny', 3640.5352682199295),
 ('zone', 3633.9040564381085),
 ('aesop', 3633.9040564381085),
 ('sonic', 3633.9040564381085),
 ('motion', 3627.2728446562869),
 ('soundtrack', 3627.2728446562869),
 ('acting', 3620.6416328744654),
 ('totally', 3607.3792093108227),
 ('gigse', 3607.3792093108227),
 ('gypsy', 3600.7479975290016),
 ('entrepreneur

In [58]:
weights['warriors']

2660.9827458384025

In [9]:
sorted(zip(train,pred_labels),key=lambda x: x[1][1],reverse=True)

[('Luke Bryan with Little Big Town and Dustin Lynch - 2 GA PIT tickets',
  ('Luke Bryan with Little Big Town and Dustin Lynch', 19745.220978556641)),
 ('LUKE BRYAN TICKETS \xe2\x98\x85 LITTLE BIG TOWN - DUSTIN LYNCH - LAST MIN DEALS!',
  ('Luke Bryan with Little Big Town and Dustin Lynch', 19745.220978556641)),
 ('Luke Bryan, Little Big Town, and Dustin Lynch Ticket',
  ('Luke Bryan with Little Big Town and Dustin Lynch', 19745.220978556641)),
 ('QUARTERFINALS: HOUSTON ROCKETS AT GOLDEN STATE WARRIORS - GAME 5 (HOME',
  ('Golden State Warriors vs. Houston Rockets Conference Quarterfinals',
   12770.347025958301)),
 ("Tony Visconti + Woody Woodmansey's HOLY HOLY -- Celebrating David Bowie",
  ("Tony Visconti + Woody Woodmansey's Holy Holy - Celebrating David Bowie",
   12330.592159945139)),
 ('San Francisco Giants vs Toronto Blue Jays (May 9)',
  ('San Francisco Giants vs. Toronto Blue Jays', 12306.794474697868)),
 ('Luke Bryan Dustin Lynch Concord Pavillion 4/29 2 Tickets Section 208',

In [10]:
[x for x in real_events if "Padres" in x]

[]

In [ ]:
#Step 2: update stop words
sw = ['tickets','']

In [104]:
sorted([(k,v) for k,v in weights.iteritems()],key=lambda x: x[1],reverse=True)

[(u'tickets', 0.4991758335277604),
 (u'beyonce', 0.2780230121466058),
 (u'for', 0.27390568065833643),
 (u'16', 0.27180836296235683),
 (u'day', 0.26968351175578803),
 (u'tix', 0.26752966999350858),
 (u'ga', 0.26534525980290108),
 (u'ticket', 0.26534525980290108),
 (u'seats', 0.26312856828006981),
 (u'row', 0.25626528808673044),
 (u'2016', 0.25626528808673044),
 (u'8pm', 0.25626528808673044),
 (u'at', 0.25626528808673044),
 (u'passes', 0.25389900903689144),
 (u'may', 0.25148918138664678),
 (u'face', 0.25148918138664678),
 (u'lower', 0.25148918138664678),
 (u'floor', 0.25148918138664678),
 (u'level', 0.24903282493903359),
 (u'theater', 0.24652663111111187),
 (u'fox', 0.24652663111111187),
 (u'49ers', 0.24396691116097483),
 (u'value', 0.24396691116097483),
 (u'club', 0.23866984780125136),
 (u'sec', 0.23592259032750496),
 (u'sold', 0.23592259032750496),
 (u'out', 0.23592259032750496),
 (u'center', 0.23310176828174942),
 (u'kaskade', 0.23020051325675128),
 (u'tour', 0.23020051325675128),
 (u

In [98]:
unlabeled = [train[i] for i,x in enumerate(pred_labels) if x[0] == None]
X = tfidf.fit_transform(vectorizer.fit_transform(unlabeled))
weights = {x[0]:x[1] for x in zip(vectorizer.get_feature_names(),tfidf.idf_)}
weighted = [(x,sumW(WORD.findall(x.lower()),weights)) for x in unlabeled]
sorted(weighted, key=lambda x: x[1],reverse=True)

[('Tori Kelly concert tickets May 19 Fox theater loge center best seats',
  2.6407417043011714),
 ('KMEL Summer Jam with Fetty Wap, YG, Rae Sremmurd, Yo Gotti, DJ Mustard',
  2.5304935031896338),
 ('KMEL Summer Jam with Fetty Wap, YG, Rae Sremmurd, Yo Gotti, DJ Mustard',
  2.5304935031896338),
 ('KMEL Summer Jam with Fetty Wap, YG, Rae Sremmurd, Yo Gotti, DJ Mustard',
  2.5304935031896338),
 ('KMEL Summer Jam with Fetty Wap, YG, Rae Sremmurd, Yo Gotti, DJ Mustard',
  2.5304935031896338),
 ('KMEL Summer Jam with Fetty Wap, YG, Rae Sremmurd, Yo Gotti, DJ Mustard',
  2.5304935031896338),
 ('KMEL Summer Jam with Fetty Wap, YG, Rae Sremmurd, Yo Gotti, DJ Mustard',
  2.5304935031896338),
 ('2 Rihanna Tickets Friday May 6 at SAP - Discount on great seats!',
  2.4637847351429278),
 ('49ers split season tickets face value - club view without club price -',
  2.3826554079366802),
 ('Guns N Roses AT&T Park Pit Tickets --- Save on Stub Hub Fees',
  2.3690554097307128),
 ('Lower Club 140 Beyonce Fo

In [14]:
#Step 4: Extract entities

#find optimal clusters
from sklearn import pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.grid_search import GridSearchCV
from spacy import en
import re

TOK = re.compile(r"(?u)\b[0-9A-Z]\w+\b")
def tokenizer(s): 
    return [x.lower() for x in TOK.findall(s)]

km = KMeans(init='k-means++', max_iter=100, n_init=1)

pipe = pipeline.Pipeline([
        #('dedup', Dedup_transformer(to_replace)),
        ('vectorizer', CountVectorizer(min_df=2,
                                       ngram_range=(1,1),
                                       stop_words=en.STOPWORDS.union(sw),
                                       tokenizer=tokenizer,
                                       lowercase=False)),
        ('transformer', ModTfidfTransformer()),
        ('kmeans',km)
    ])
#142
params = dict(kmeans__n_clusters= range(244,300))

grid = GridSearchCV(pipe,param_grid=params,n_jobs=-1)

grid.fit(cleaner)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=False, max_df=1.0, max_features=None, min_df=2,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=s...n_init=1,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'kmeans__n_clusters': [244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [15]:
grid.best_params_

{'kmeans__n_clusters': 299}

In [121]:
idfs = grid.best_estimator_.named_steps['transformer'].idf_
a = []
for w, i in grid.best_estimator_.named_steps['vectorizer'].vocabulary_.iteritems():
    a.append((w,idfs[i]))

sorted(a,key=lambda x: x[1],reverse=True)

[(u'coachella', 0.35978462498556313),
 (u'warriors', 0.35824834640534808),
 (u'weekend', 0.32452331046862748),
 (u'giants', 0.3163070068108027),
 (u'playoff', 0.27282221746342816),
 (u'grizzlies', 0.26266256194062765),
 (u'game', 0.25801883577523538),
 (u'shuttle', 0.25754586804882856),
 (u'sf', 0.25563737644984824),
 (u'ga', 0.25418794965210595),
 (u'pass', 0.24872992987539991),
 (u'jack', 0.2430183424693472),
 (u'san', 0.24248505090981273),
 (u'sharks', 0.2364442894467752),
 (u'golden', 0.23416083215207315),
 (u'playoffs', 0.23358203559928881),
 (u'state', 0.23358203559928881),
 (u'round', 0.23123331992629445),
 (u'club', 0.22760227819615428),
 (u'passes', 0.22698356449702553),
 (u'box', 0.22698356449702553),
 (u'sec', 0.2218791567456711),
 (u'beyonce', 0.21853382346129802),
 (u'selling', 0.21288773168286096),
 (u'april', 0.21288773168286096),
 (u'oakland', 0.21215332784742846),
 (u'fox', 0.21141195971540871),
 (u'day', 0.21141195971540871),
 (u'jose', 0.20914399886905807),
 (u'face'

In [20]:
# get top terms per cluster
order_centroids = grid.best_estimator_.named_steps['kmeans'].cluster_centers_.argsort()[:, ::-1]
terms = grid.best_estimator_.named_steps['vectorizer'].get_feature_names()
for i in range(299):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :5]:
        print(' %s' % terms[ind])
    print()

Cluster 0:
 david
 adam
 sandler
 spade
 floor
()
Cluster 1:
 warriors
 grizzlies
 memphis
 sec
 secti
()
Cluster 2:
 coachella
 weekend
 ga
 car
 camping
()
Cluster 3:
 giants
 sf
 event
 special
 marlins
()
Cluster 4:
 jack
 ticketmaster
 french
 game
 ga
()
Cluster 5:
 playoff
 sharks
 great
 face
 sj
()
Cluster 6:
 day
 outside
 pass
 lands
 passes
()
Cluster 7:
 fox
 face
 value
 theater
 m83
()
Cluster 8:
 warriors
 playoff
 round
 game
 sec
()
Cluster 9:
 giants
 sec
 games
 chicago
 plb
()
Cluster 10:
 beyonce
 vip
 tour
 formation
 world
()
Cluster 11:
 sec
 acoustic
 zhu
 fresno
 game
()
Cluster 12:
 giants
 san
 club
 francisco
 season
()
Cluster 13:
 magazine
 pop
 sold
 tonight
 orchestra
()
Cluster 14:
 warriors
 sec
 play
 san
 round
()
Cluster 15:
 coachella
 weekend
 vip
 sale
 wristband
()
Cluster 16:
 ballet
 mana
 international
 bryan
 russell
()
Cluster 17:
 jack
 graham
 tuesday
 ga
 auditorium
()
Cluster 18:
 coachella
 shuttle
 pass
 wknd
 admission
()
Cluster 1

In [1]:
%%time
#import data
import requests 
from bs4 import BeautifulSoup 
import re

class Scrape:
    def parsed_posts(self, soup_p):
        titles = []

        for post in soup_p:
            titles.append(post.find(class_='hdrlnk').contents[0])
        return titles

    def run(self):
        
        #all data
        agg_posts = []


        current_post = 0

        baseURL='https://sfbay.craigslist.org/search/tia?'

        URL = baseURL + 's=0'
        print 'getting request...' + URL
        response = requests.get(URL)
        print 'OK'
        html = response.content
        soup = BeautifulSoup(html)
        soup_posts = soup.findAll('p')

        while len(soup_posts) > 0:
            agg_posts.extend(self.parsed_posts(soup_posts))
            current_post += 100
            URL = baseURL + '&s=' + str(current_post)
            print 'getting request...' + URL
            response = requests.get(URL)
            print 'OK'
            html = response.content
            soup = BeautifulSoup(html)
            soup_posts = soup.findAll('p')
            print 'page: ' + str(current_post)

        path = 'training.txt'
        with open(path,'w') as f:
            f.writelines([unicode(x.contents[0]+'\n').encode('utf8') for x in agg_posts])
        
        
if __name__ == '__main__':
    a = Scrape()
    a.run()

getting request...https://sfbay.craigslist.org/search/tia?s=0
OK
getting request...https://sfbay.craigslist.org/search/tia?&s=100
OK
page: 100
getting request...https://sfbay.craigslist.org/search/tia?&s=200
OK
page: 200
getting request...https://sfbay.craigslist.org/search/tia?&s=300
OK
page: 300
getting request...https://sfbay.craigslist.org/search/tia?&s=400
OK
page: 400
getting request...https://sfbay.craigslist.org/search/tia?&s=500
OK
page: 500
getting request...https://sfbay.craigslist.org/search/tia?&s=600
OK
page: 600
getting request...https://sfbay.craigslist.org/search/tia?&s=700
OK
page: 700
getting request...https://sfbay.craigslist.org/search/tia?&s=800
OK
page: 800
getting request...https://sfbay.craigslist.org/search/tia?&s=900
OK
page: 900
getting request...https://sfbay.craigslist.org/search/tia?&s=1000
OK
page: 1000
getting request...https://sfbay.craigslist.org/search/tia?&s=1100
OK
page: 1100
getting request...https://sfbay.craigslist.org/search/tia?&s=1200
OK
page

In [1]:
from spacy.en import English
nlp = English()

In [62]:
datetime.strptime('20160506','%Y%m%d').strftime("%A, %d %B %Y")

'Friday, 06 May 2016'

In [7]:
s = 'tickets for Coachella on Wednesday'
path = '/home/vagrant/datacourse/fellow_only_material/Capstone/'
def extract():
    #from spacy.en import English
    #nlp = English()
    print 'quit to close.'
    threshold = 0.1
    model = Sim_predictor()
    print 'Fitting model...'
    sw = en.STOPWORDS.union(set(['ticket','tickets','san','francisco','s']))
    model.fit(real_events, sw,0.01)
    spl = re.compile(r'(.*)\s+(\d{8})\s+(.*)')
    while True:
        text = raw_input('Text to parse: ')
        if text == 'quit': break
        to_print = model.predict([text],threshold)[0][1][0]
        if to_print:
            info = spl.match(to_print)
            if info:
                g = info.groups()
                print 'What: ' + g[0]
                print 'When: ' + datetime.strptime(g[1],'%Y%m%d').strftime("%A, %d %B %Y")
                print 'Where: ' + g[2]
            else:
                print 'Not found'
        else:
            print 'Not found'
      
if __name__=='__main__':
    extract()

quit to close.
Fitting model...
Text to parse: A's vs Rockets
What: Golden State Warriors vs. Houston Rockets Conference Quarterfinals
When: Sunday, 01 May 2016
Where: Oracle Arena
Text to parse: Wanna see new Adam’s Sandler show?
What: Adam Sandler, David Spade, Nick Swardson & Rob Schneider
When: Thursday, 19 May 2016
Where: Bill Graham Civic Auditorium


KeyboardInterrupt: 

In [209]:
752.0/100 *907169

6821910.88